In [23]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [15]:
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

TensorFlow version is  1.13.1


In [43]:
y_train = pd.read_csv('Data/Train/trainLbls.csv',names = ['label'])
y_train['label'] = y_train['label'].astype(str)
y_train['id'] = pd.Series(np.arange(1,len(y_train)+1))

y_validation = pd.read_csv('Data/Validation/valLbls.csv', names = ['label'])
y_validation['label'] = y_validation['label'].astype(str)
y_validation['id'] = pd.Series(np.arange(1,len(y_validation)+1))

In [44]:
image_size = 160 # All images will be resized to 160x160
batch_size = 32
IMG_SHAPE = (image_size, image_size, 3)

train_dir = 'Data/Train/TrainImages/'
validation_dir = 'Data/Validation/ValidationImages/'

def append_ext(fn):
    return 'Image'+str(fn)+'.jpg'
  
y_train['id']=y_train['id'].apply(append_ext)
y_validation['id']=y_validation['id'].apply(append_ext)
y_validation.shape

(2298, 2)

In [45]:
# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(
                y_train,
                train_dir,  # Source directory for the training images 
                x_col="id",
                y_col="label",
                target_size=(image_size, image_size),  
                batch_size=batch_size)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(
                y_validation,
                validation_dir, # Source directory for the validation images 
                x_col="id",
                y_col="label",
                target_size=(image_size, image_size),
                batch_size=batch_size)

Found 5830 images belonging to 29 classes.
Found 2298 images belonging to 29 classes.


In [46]:
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')

In [47]:
base_model.trainable = False

In [48]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 79, 79, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_282 (Bat (None, 79, 79, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 79, 79, 32)   0           batch_normalization_v1_282[0][0] 
__________________________________________________________________________________________________
conv2d_283

In [49]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(30, activation='softmax')
])

In [50]:
model.compile(optimizer="adam", 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                61470     
Total params: 21,864,254
Trainable params: 61,470
Non-trainable params: 21,802,784
_________________________________________________________________


In [52]:
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)

Epoch 1/10


InvalidArgumentError: Can not squeeze dim[1], expected a dimension of 1, got 29
	 [[{{node metrics_2/acc/Squeeze}}]]